# Introduction to Spark programming

# Word Count

The "Hello World!" of distributed programming is the wordcount. Basically, you want to count easily number of different words contained in an unstructured text. You will write some code to perform this task on the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) retrieved from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page).

[Spark's Python API reference](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) could provide some help

### ** Part 1: Creating a base RDD and pair RDDs **

#### In this part of the lab, we will explore creating a base RDD with `parallelize` and using pair RDDs to count words.

#### We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

In [0]:
#Please, run this first
import hashlib
import sys

def hash(x):
  return hashlib.sha1(str(x).encode('utf-8')).hexdigest()

assert sys.version_info.major == 3

In [0]:
words_list = ['we', 'few', 'we', 'happy', 'few', "we", "band", "of", "brothers"]
words_RDD = sc.parallelize(words_list, 4)

# Print the type of words_RDD
print(type(words_RDD))

<class 'pyspark.rdd.RDD'>


We want to capitalize each word contained in a RDD. For such transformation, we use a `map`, as we want to transform a RDD of **n** elements into another RDD of **n** using a function that gets and returns one single element.

Please implement `capitalize`function in the cell below.

In [0]:
def capitalize(word):
    """Capitalize lowercase `words`.

    Args:
        word (str): A lowercase string.

    Returns:
        str: A string which first letter is uppercase.
    """
    return word[0].upper() + word[1:]

print(capitalize('we'))

We


Apply `capitalize` to the base RDD, using a [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) transformation that applies the `capitalize()` function to each element. Then call the [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) action to retrieve the values of the transformed RDD, and print them.

In [0]:
local_result = words_RDD.map(capitalize).collect()

print(local_result)

assert hash(local_result) == 'bd73c54004cc9655159aceb703bc14fe93369fb1',\
       'Capitalize'

['We', 'Few', 'We', 'Happy', 'Few', 'We', 'Band', 'Of', 'Brothers']


Do the same using a lambda function

In [0]:
local_result = words_RDD.map(lambda x: x.capitalize()).collect()

print(local_result)

assert hash(local_result) == 'bd73c54004cc9655159aceb703bc14fe93369fb1',\
       'Capitalize with lambda'

['We', 'Few', 'We', 'Happy', 'Few', 'We', 'Band', 'Of', 'Brothers']


Now use `map()` and a `lambda` function to return the number of characters in each word, and `collect` this result directly into a variable.

In [0]:
plural_lengths = (words_RDD.map(lambda x: len(x)).collect())

print(plural_lengths)

assert hash(plural_lengths) == '0772853c8e180c1bed8cfe9bde35aae79b277381',\
       'incorrect values for plural_lengths'

[2, 3, 2, 5, 3, 2, 4, 2, 8]


To program a wordcount, we will need `pair RDD` objects. A pair RDD is an RDD where each element is a pair tuple `(k, v)` where `k` is the key and `v` is the value. In this example, we will create a pair consisting of `('<word>', 1)` for each word element in the RDD.

Create the pair RDD using the `map()` transformation with a `lambda()` on `words_RDD`.

In [0]:
words_pair_RDD = words_RDD.map(lambda x: (x, 1))

print(words_pair_RDD.collect())

assert hash(words_pair_RDD.collect()) == 'fb67a530034e01395386569ef29bf5565b503ec6', \
       'incorrect value for words_pair_RDD'

[('we', 1), ('few', 1), ('we', 1), ('happy', 1), ('few', 1), ('we', 1), ('band', 1), ('of', 1), ('brothers', 1)]


Now, let's count the number of times a particular word appears in the RDD. There are multiple ways to perform the counting, but some are much less efficient or scalable than others.

A naive approach would be to `collect()` all of the elements and count them in the driver program. While this approach could work for small datasets, it is not scalable as the result of `collect()` would have to fit in the driver's memory. When you should use `collect()` with care, always asking yourself what is the size of data you want to retrieve.

In order to program a scalable wordcount, you will need to use parallel operations.

#### `groupByKey()` approach

An approach you might first consider is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. This transformation groups all the elements of the RDD with the same key into a single list, stored in one of the partitions. 
 
Use `groupByKey()` on `words_pair_RDD`
 to generate a pair RDD of type `('word', list)`.

In [0]:
words_grouped = words_pair_RDD.groupByKey()

for key, value in words_grouped.collect():
    print('{0}: {1}'.format(key, list(value)))
    
assert hash(sorted(words_grouped.mapValues(lambda x: list(x)).collect())) == 'fdaad77fd81ef2df23d98ff7fd438fa700ca1fcf',\
       'incorrect value for words_grouped'

of: [1]
few: [1, 1]
brothers: [1]
we: [1, 1, 1]
band: [1]
happy: [1]


Using the `groupByKey()` transformation results in an `pairRDD` containing words as keys, and Python iterators as values. Python iterators are a class of objects on which we can iterate, i.e.

    a = some_iterator()
    for elem in a:
        # do stuff with elem

Python lists and dictionnaries are iterators for example.

Now sum the iterator using a `map()` transformation. The result should be a pair RDD consisting of (word, count) pairs.

Hint: there exists a `sum` function
Hint 2: you want to perform an operation only on the values of the pairRDD. Take a look at [mapValues()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.mapValues).

In [0]:
word_grouped_counts = words_grouped.mapValues(lambda x: sum(x))

print(word_grouped_counts.collect())

assert hash(sorted(word_grouped_counts.collect())) == 'c20f05d36e98ae399b2cbe5b6cb9bf01b675455a',\
       'incorrect value for word_grouped_counts'

[('of', 1), ('few', 2), ('brothers', 1), ('we', 3), ('band', 1), ('happy', 1)]


There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions (remember the cost of network communications!).
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory of a worker.

A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

Compute the word count using `reduceByKey`

In [0]:
from operator import add
word_counts = words_pair_RDD.reduceByKey(add)

print(word_counts.collect())

assert hash(sorted(word_counts.collect())) == 'c20f05d36e98ae399b2cbe5b6cb9bf01b675455a',\
       'incorrect value for word_counts'

[('of', 1), ('few', 2), ('brothers', 1), ('we', 3), ('band', 1), ('happy', 1)]


You should be able to perform the word count by composing functions, resulting in a smaller code. Use the `map()` on word RDD to create a pair RDD, apply the `reduceByKey()` transformation, and `collect` in one statement.

In [0]:
word_counts_collected = (words_RDD
                         .map(lambda x: (x, 1))
                         .reduceByKey(add)
                         .collect()
                        )

print(word_counts_collected)

assert hash(sorted(word_counts_collected)) == 'c20f05d36e98ae399b2cbe5b6cb9bf01b675455a', \
       'incorrect value for word_counts_collected'

[('of', 1), ('few', 2), ('brothers', 1), ('we', 3), ('band', 1), ('happy', 1)]


Compute the number of unique words using one of the RDD you have already created.

In [0]:
unique_words = words_RDD.distinct().count()
#unique_words = len(word_counts_collected)

print(unique_words)

assert unique_words == 6, 'incorrect count of unique_words'

6


Use a `reduce()` action to sum the counts in `wordCounts` and then divide by the number of unique words to find the mean number of words per unique word in `word_counts`.  First `map()` RDD `word_counts`, which consists of (key, value) pairs, to an RDD of values.

In [0]:
total_count = (word_counts
              .map(lambda x: x[1])
              .reduce(add))

average = total_count / float(unique_words)

print(total_count)
print(round(average, 2))

assert round(average, 2) == 1.5, 'incorrect value of average'

9
1.5


## Part 2: Apply word count to a file

In this section we will finish developing our word count application.  We'll have to build the `word_count` function, deal with real world problems like capitalization and punctuation, load in our data source, and compute the word count on the new data.

First, define a function for word counting. You should reuse the techniques that have been covered in earlier parts of this lab.  This function should take in an RDD that is a list of words like `words_RDD` and return a pair RDD that has all of the words and their associated counts.

In [0]:
def word_count(word_list_RDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    pair_RDD = word_list_RDD.map(lambda x: (x, 1)).reduceByKey(add)
    return pair_RDD

print(word_count(words_RDD).collect())

assert hash(sorted(word_count(words_RDD).collect())) == 'c20f05d36e98ae399b2cbe5b6cb9bf01b675455a', \
       'incorrect definition for word_count function'

[('of', 1), ('few', 2), ('brothers', 1), ('we', 3), ('band', 1), ('happy', 1)]


Real world data is more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.
 
Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.

If you have never used regex (regular expressions) before, you can refer to [Regular-expressions.info](http://www.regular-expressions.info/python.html)

In order to test your regular expressions, you can use [Regex Tester](http://www.regexpal.com)

Regex can be a bit obscure at beginning, don't hesitate to search in [StackOverflow](http://stackoverflow.com) or to ask me for some help.

In [0]:
import re
import string

# Hint: string.punctuation contains all the punctuation symbols

def remove_punctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text) # removes punctuation
    text = text.replace("_", "")
    text = text.strip()
    
    return text

print(remove_punctuation('Hello World!'))
print(remove_punctuation(' No under_score!'))

assert remove_punctuation("  Remove punctuation: there ARE trailing spaces. ") == 'remove punctuation there are trailing spaces', \
       'incorrect definition for remove_punctuation function'

hello world
no underscore


For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `remove_punctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lowercase.  Since the file is large we use `take(15)`, instead of `collect()` so that we only print 15 lines.

Take a look at [zipWithIndex()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.zipWithIndex) and [take()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take) to understand the print statement.

To understand how we load the data, look at [databricks documentation](https://docs.databricks.com/user-guide/importing-data.html)

In [0]:
%sh wget https://waterponey.github.io/BigDataClass/files/shakespeare.txt -O shakespeare.txt

--2022-01-10 17:09:33--  https://waterponey.github.io/BigDataClass/files/shakespeare.txt
Resolving waterponey.github.io (waterponey.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to waterponey.github.io (waterponey.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5589889 (5.3M) [text/plain]
Saving to: ‘shakespeare.txt’

     0K .......... .......... .......... .......... ..........  0% 6.12M 1s
    50K .......... .......... .......... .......... ..........  1% 4.37M 1s
   100K .......... .......... .......... .......... ..........  2% 6.23M 1s
   150K .......... .......... .......... .......... ..........  3% 17.5M 1s
   200K .......... .......... .......... .......... ..........  4%  120M 1s
   250K .......... .......... .......... .......... ..........  5% 75.2M 1s
   300K .......... .......... .......... .......... ..........  6% 10.8M 1s
   350K .......... .......... .......... .......... .........

In [0]:
file_path = 'file:/databricks/driver/shakespeare.txt'

shakespeare_RDD = (sc.textFile(file_path, 8)
                     .map(remove_punctuation))

print('\n'.join(shakespeare_RDD
                .zipWithIndex()  # to (line, lineNum) pairRDD
                .map(lambda pair: '{0}: {1}'.format(pair[1], pair[0]))  # to 'lineNum: line'
                .take(15)))

0: the project gutenberg ebook of the complete works of william shakespeare by
1: william shakespeare
2: 
3: this ebook is for the use of anyone anywhere at no cost and with
4: almost no restrictions whatsoever  you may copy it give it away or
5: reuse it under the terms of the project gutenberg license included
6: with this ebook or online at wwwgutenbergorg
7: 
8: this is a copyrighted project gutenberg ebook details below
9: please follow the copyright guidelines in this file
10: 
11: title the complete works of william shakespeare
12: 
13: author william shakespeare
14: 


Before we can use the `word_count()` function, we have to address two issues with the format of the RDD:
  + #### The first issue is that  that we need to split each line by its spaces.
  + #### The second issue is we need to filter out empty lines.
 
Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/2/library/string.html#string.split) function. You might think that a [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) transformation is the way to do this, but think about what the result of the `split()` function will be: there is a better option.

Hint: remember the problem we had with `GroupByKey()`

In [0]:
shakespeare_words_RDD = shakespeare_RDD.flatMap(lambda x: x.split())
shakespeare_word_count_elem = shakespeare_words_RDD.count()

print(shakespeare_words_RDD.take(5))
print(shakespeare_word_count_elem)

#assert shakespeare_word_count_elem == 1010679, \
#       'incorrect value for shakespeare_word_count_elem'

#assert hash(shakespeare_words_RDD.top(5)) == '036f886dad6af36a651fd51910180a9a0e267d84', \
#       'incorrect value for shakespeare_words_RDD'

['the', 'project', 'gutenberg', 'ebook', 'of']
903705


The next step is to filter out the empty elements.  Remove all entries where the word is `''`.

In [0]:
shakespeare_nonempty_words_RDD = shakespeare_words_RDD.filter(lambda x: x != "")
shakespeare_nonempty_word_elem_count = shakespeare_nonempty_words_RDD.count()

print(shakespeare_nonempty_word_elem_count)

#assert shakespeare_nonempty_word_elem_count == 960028, \
#       'incorrect value for shakespeare_nonempty_word_elem_count'

903705


You now have an RDD that contains only words.  Next, apply the `word_count()` function to produce a list of word counts. We can view the top 15 words by using the `takeOrdered()` action. However, since the elements of the RDD are pairs, you will need a custom sort function that sorts using the value part of the pair.

Use the `wordCount()` function and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [0]:
top15_words = shakespeare_nonempty_words_RDD.map(lambda x: word_count(x)).takeOrdered(15)

print('\n'.join(map(lambda pair: '{0}: {1}'.format(pair[0], pair[1]), top15_words)))

assert hash(top15_words) == '3c4a0974cc09536ba8902683e0ad441d8239a710', \
       'incorrect value for top15WordsAndCounts'

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-4229865841361056> in <module>
----> 1 top15_words = shakespeare_nonempty_words_RDD.flatMap(lambda x: word_count(x)).takeOrdered(15)
      2 
      3 print('\n'.join(map(lambda pair: '{0}: {1}'.format(pair[0], pair[1]), top15_words)))
      4 
      5 assert hash(top15_words) == '3c4a0974cc09536ba8902683e0ad441d8239a710', \

/databricks/spark/python/pyspark/rdd.py in takeOrdered(self, num, key)
   1539             return heapq.nsmallest(num, a + b, key)
   1540 
-> 1541         return self.mapPartitions(lambda it: [heapq.nsmallest(num, it, key)]).reduce(merge)
   1542 
   1543     def take(self, num):

/databricks/spark/python/pyspark/rdd.py in reduce(self, f)
   1030             yield reduce(f, iterator, initial)
   1031 
-> 1032         vals = self.mapPartitions(func).collect()
   1033         if vals:
   1034             retu

You will notice that many of the words are common English words. These are called stopwords. In practice, when we do natural language processing, we filter these stopwords as they do not contain a lot of information.